In [ ]:
# ==============================================================================
# CÉLULA 1: Configuração do Ambiente e Funções Auxiliares
# ==============================================================================
# Descrição: Importa bibliotecas, define constantes globais (datas, caminhos),
# e cria funções de utilidade que serão usadas ao longo do notebook.
# ==============================================================================
import pandas as pd
import requests
import zipfile
import warnings
import re
from pathlib import Path
import DataUtils as du
import time

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [ ]:
# --- Configurações de Datas e Diretórios ---
# Use datas fixas para reprodutibilidade ou ajuste para datas dinâmicas.
# Ex: end=pd.Timestamp.now().strftime('%Y-%m-%d')
DATE_START = '2024-01-01'
DATE_END   = '2024-12-31'

date_range_monthly = pd.date_range(start=DATE_START, end=DATE_END, freq='M').strftime("%Y%m").tolist()
date_range_quarterly = pd.date_range(start=DATE_START, end=DATE_END, freq='Q').strftime("%Y%m").tolist()

# Estrutura de diretórios
base_dir    = Path('..').resolve() # O notebook está na raiz do projeto ou em 'Code'
dir_inputs  = base_dir / 'Input'
dir_outputs = base_dir / 'Output'
for d in (dir_inputs, dir_outputs):
    d.mkdir(parents=True, exist_ok=True)

print(f"Diretório Base: {base_dir}")
print(f"Diretório de Inputs: {dir_inputs}")
print(f"Diretório de Outputs: {dir_outputs}")
print(f"\nPeríodo de Análise Mensal: de {date_range_monthly[0]} a {date_range_monthly[-1]}")
print(f"Período de Análise Trimestral: de {date_range_quarterly[0]} a {date_range_quarterly[-1]}")

In [ ]:
# --- Funções Auxiliares Locais ---

def clean_text_column(series: pd.Series) -> pd.Series:
    """Limpa uma coluna de texto, removendo caracteres de controle e espaços extras."""
    if series.empty:
        return series
    
    def remove_illegal_chars(text: str) -> str:
        if pd.isna(text): return ''
        # Remove caracteres de controle ASCII (exceto tab, newline, etc. que já são tratados pelo strip/replace)
        return re.sub(r'[\x00-\x1F\x7F]', '', str(text))

    return series.astype(str).apply(remove_illegal_chars).str.replace(r'\s+', ' ', regex=True).str.strip()

def create_data_infos(df: pd.DataFrame, df_name: str) -> pd.DataFrame:
    """Cria um perfil de dados (info) a partir de um DataFrame."""
    print(f"Gerando perfil do DataFrame: '{df_name}'...")
    
    dict_df = pd.DataFrame({
        'Coluna': df.columns,
        'Tipo de Dado (Dtype)': df.dtypes.astype(str),
        'Valores Não Nulos': df.count().values,
        'Valores Nulos': df.isnull().sum().values,
    })
    dict_df['% Nulos'] = (dict_df['Valores Nulos'] / len(df) * 100).round(2)
    
    def get_example(col):
        try:
            return df[col].dropna().unique()[:3]
        except Exception:
            return "Não foi possível extrair exemplos"
            
    dict_df['Exemplos'] = [get_example(col) for col in df.columns]
    
    return dict_df

In [ ]:
# ==============================================================================
# CÉLULA 2: Pipeline de Dados - COSIF Individual
# ==============================================================================
# Descrição: Baixa, extrai, lê e consolida os arquivos mensais do
# COSIF Individual (Bancos). O resultado final é um único DataFrame
# salvo em formato Parquet com nomes de colunas padronizados.
# ==============================================================================
print("\n>>> INICIANDO PIPELINE: COSIF INDIVIDUAL <<<")

# --- 1. Download e Extração dos Arquivos ---
cosif_ind_dir = dir_inputs / 'COSIF' / 'individual'
cosif_ind_dir.mkdir(parents=True, exist_ok=True)
all_individual_csv_paths = []

suffixes = ['BANCOS.csv.zip', 'BANCOS.zip' , 'BANCOS.csv']

for date in date_range_monthly:
    subfolder = cosif_ind_dir / date
    subfolder.mkdir(exist_ok=True)
    
    existing_csvs = list(subfolder.glob("*BANCOS*.csv"))
    if existing_csvs:
        print(f"COSIF Ind. ({date}): CSV já existe, pulando download.")
        all_individual_csv_paths.extend(existing_csvs)
        continue

    download_success = False
    for suffix in suffixes:
        if download_success:
            break # Se já baixou para esta data, sai do loop de sufixos

        local_file = subfolder / f"{date}{suffix}"

        url = f"https://www.bcb.gov.br/content/estabilidadefinanceira/cosif/Bancos/{date}{suffix}"

        print(f"COSIF Ind. ({date}): Tentando baixar de {url}...")
        try:
            resp = requests.get(url, timeout=90)
            resp.raise_for_status() # Lança erro para status 4xx/5xx

            content_type = resp.headers.get('Content-Type', '').lower()
            # Validação para garantir que não é uma página de erro HTML
            if 'html' in content_type:
                print("  -> Falha: Recebido conteúdo HTML, pulando para próximo sufixo.")
                continue

            # --- Lógica para tratar ZIP ---
            if 'zip' in suffix.lower():
                with open(local_file, 'wb') as f:
                    f.write(resp.content)
                
                with zipfile.ZipFile(local_file, 'r') as zf:
                    extracted = False
                    for member in zf.namelist():
                        if member.lower().endswith('.csv') and "bancos" in member.lower():
                            zf.extract(member, subfolder)
                            extracted_file = subfolder / member
                            all_individual_csv_paths.append(extracted_file)
                            print(f"  -> Sucesso! Arquivo '{member}' extraído de '{local_file.name}'.")
                            extracted = True
                    if extracted:
                        download_success = True # Marca o sucesso para esta data
                    else:
                        print("  -> Falha: ZIP baixado não contém o CSV esperado.")

            # --- Lógica para tratar CSV direto ---
            elif '.csv' in suffix.lower():
                # Salva o arquivo CSV diretamente
                with open(local_file, 'wb') as f:
                    f.write(resp.content)
                all_individual_csv_paths.append(local_file)
                print(f"  -> Sucesso! Arquivo CSV '{local_file.name}' baixado diretamente.")
                download_success = True # Marca o sucesso para esta data

        except requests.exceptions.HTTPError as e:
            # Erro comum para arquivo não encontrado (404), não é um erro fatal do script
            print(f"  -> Info: Arquivo não encontrado em {url} (Status: {e.response.status_code}).")
        except requests.exceptions.RequestException as e:
            print(f"  -> Erro de conexão ao tentar {url}: {e}")
        except zipfile.BadZipFile:
            print(f"  -> Erro: Arquivo baixado de {url} não é um ZIP válido.")
        except Exception as e:
            print(f"  -> Erro inesperado ao processar {url}: {e}")

    # Se após todas as tentativas o download falhou para a data
    if not download_success:
        print(f"COSIF Ind. ({date}): FALHA no download após tentar todos os sufixos.")

# --- 2. Leitura e Consolidação dos CSVs ---
df_list = []
print(f"\nLendo {len(all_individual_csv_paths)} arquivo(s) CSV do COSIF Individual...")
for csv_path in sorted(list(set(all_individual_csv_paths))):
    print(f"  Lendo: {csv_path.name}")
    try:
        temp_df = pd.read_csv(csv_path, header=3, encoding='latin1', sep=';', decimal=',', dtype={'CNPJ': str}, on_bad_lines='warn')
        file_date = csv_path.parent.name
        temp_df['DATA'] = file_date
        df_list.append(temp_df)
    except Exception as e:
        print(f"    -> Erro ao ler o arquivo {csv_path.name}: {e}")

# --- 3. Processamento e Limpeza do DataFrame ---
if not df_list:
    print("\nAVISO: Nenhum dado do COSIF Individual foi carregado. O DataFrame final estará vazio.")
    df_cosif_individual = pd.DataFrame()
else:
    df_cosif_individual = pd.concat(df_list, ignore_index=True)
    print(f"\nShape do DF consolidado (bruto): {df_cosif_individual.shape}")

    # Padronização da chave CNPJ_8
    df_cosif_individual['CNPJ_8'] = du.standardize_cnpj_base8(df_cosif_individual['CNPJ'])
    
    # Padronização de data
    df_cosif_individual['DATA'] = pd.to_datetime(df_cosif_individual['DATA'], format='%Y%m').dt.strftime('%Y%m').astype(int)

    # Mapa de renomeação para o novo padrão
    rename_map = {
        'NOME_INSTITUICAO': 'NOME_INSTITUICAO_COSIF',
        'CONTA': 'CONTA_COSIF',
        'NOME_CONTA': 'NOME_CONTA_COSIF',
        'SALDO': 'VALOR_CONTA_COSIF',
        'COD_CONGL': 'COD_CONGL_PRUD_COSIF',
        'NOME_CONGL': 'NOME_CONGL_PRUD_COSIF',
        'TAXONOMIA': 'TAXONOMIA_COSIF',
        'DOCUMENTO': 'DOCUMENTO_COSIF',
        'AGENCIA': 'AGENCIA_COSIF'
    }
    df_cosif_individual.rename(columns=rename_map, inplace=True)

    # Limpeza das colunas de texto (já com os novos nomes)
    for col in ['NOME_INSTITUICAO_COSIF', 'NOME_CONTA_COSIF', 'TAXONOMIA_COSIF']:
        if col in df_cosif_individual.columns:
            df_cosif_individual[col] = clean_text_column(df_cosif_individual[col])

    # Descarte de colunas redundantes
    cols_to_drop = ['CNPJ', '#DATA_BASE']
    df_cosif_individual.drop(columns=[c for c in cols_to_drop if c in df_cosif_individual.columns], inplace=True)

    # Reordenar colunas para melhor visualização, garantindo que todas as outras sejam mantidas
    cols_ordem_prioritaria = [
        'DATA', 'CNPJ_8', 'NOME_INSTITUICAO_COSIF', 'CONTA_COSIF', 'NOME_CONTA_COSIF', 
        'VALOR_CONTA_COSIF', 'COD_CONGL_PRUD_COSIF', 'NOME_CONGL_PRUD_COSIF', 
        'TAXONOMIA_COSIF', 'DOCUMENTO_COSIF', 'AGENCIA_COSIF'
    ]
    cols_existentes_prioritarias = [c for c in cols_ordem_prioritaria if c in df_cosif_individual.columns]
    cols_restantes = [c for c in df_cosif_individual.columns if c not in cols_existentes_prioritarias]
    df_cosif_individual = df_cosif_individual[cols_existentes_prioritarias + cols_restantes]

    print("\n--- Informações do DataFrame Final: df_cosif_individual ---")
    print(f"Shape: {df_cosif_individual.shape}")
    print(f"Colunas: {df_cosif_individual.columns.tolist()}")
    print(f"Período de dados: {df_cosif_individual['DATA'].min()} a {df_cosif_individual['DATA'].max()}")

    # --- 4. Salvando o Resultado ---
    output_path = dir_outputs / 'df_cosif_individual.parquet'
    df_cosif_individual.to_parquet(output_path, index=False)
    print(f"\nArquivo salvo com sucesso em: {output_path}")

In [ ]:
# ==============================================================================
# CÉLULA 3: Pipeline de Dados - COSIF Prudencial
# ==============================================================================
# Descrição: Baixa, extrai, lê e consolida os arquivos mensais do
# COSIF Prudencial (Conglomerados). O resultado final é um único DataFrame
# salvo em formato Parquet com nomes de colunas padronizados.
# ==============================================================================
print("\n>>> INICIANDO PIPELINE: COSIF PRUDENCIAL <<<")

# --- 1. Download e Extração dos Arquivos ---
cosif_prud_dir = dir_inputs / 'COSIF' / 'prudencial'
cosif_prud_dir.mkdir(parents=True, exist_ok=True)
all_prudencial_csv_paths = []

suffixes = ['BLOPRUDENCIAL.csv.zip', 'BLOPRUDENCIAL.zip', 'BLOPRUDENCIAL.csv']

for date in date_range_monthly:
    subfolder = cosif_prud_dir / date
    subfolder.mkdir(exist_ok=True)
    
    existing_csvs = list(subfolder.glob("*BLOPRUDENCIAL*.csv"))
    if existing_csvs:
        print(f"COSIF Prud. ({date}): CSV já existe, pulando download.")
        all_prudencial_csv_paths.extend(existing_csvs)
        continue

    download_success = False
    for suffix in suffixes:
        if download_success:
            break # Se já baixou para esta data, sai do loop de sufixos

        local_file = subfolder / f"{date}{suffix}"

        ### URL base para o COSIF Prudencial
        url = f"https://www.bcb.gov.br/content/estabilidadefinanceira/cosif/Conglomerados-prudenciais/{date}{suffix}"

        print(f"COSIF Prud. ({date}): Tentando baixar de {url}...")
        try:
            resp = requests.get(url, timeout=90)
            resp.raise_for_status() # Lança erro para status 4xx/5xx

            content_type = resp.headers.get('Content-Type', '').lower()
            # Validação para garantir que não é uma página de erro HTML
            if 'html' in content_type:
                print("  -> Falha: Recebido conteúdo HTML, pulando para próximo sufixo.")
                continue

            # --- Lógica para tratar ZIP ---
            if 'zip' in suffix.lower():
                with open(local_file, 'wb') as f:
                    f.write(resp.content)
                
                with zipfile.ZipFile(local_file, 'r') as zf:
                    extracted = False
                    for member in zf.namelist():
                        ### ALTERAÇÃO: Verificação do nome do arquivo interno específica para Prudencial
                        if member.lower().endswith('.csv') and "prudencial" in member.lower():
                            zf.extract(member, subfolder)
                            extracted_file = subfolder / member
                            all_prudencial_csv_paths.append(extracted_file)
                            print(f"  -> Sucesso! Arquivo '{member}' extraído de '{local_file.name}'.")
                            extracted = True
                    if extracted:
                        download_success = True # Marca o sucesso para esta data
                    else:
                        print("  -> Falha: ZIP baixado não contém o CSV esperado.")

            # --- Lógica para tratar CSV direto ---
            elif '.csv' in suffix.lower():
                # Salva o arquivo CSV diretamente
                with open(local_file, 'wb') as f:
                    f.write(resp.content)
                all_prudencial_csv_paths.append(local_file)
                print(f"  -> Sucesso! Arquivo CSV '{local_file.name}' baixado diretamente.")
                download_success = True # Marca o sucesso para esta data

        except requests.exceptions.HTTPError as e:
            # Erro comum para arquivo não encontrado (404), não é um erro fatal do script
            print(f"  -> Info: Arquivo não encontrado em {url} (Status: {e.response.status_code}).")
        except requests.exceptions.RequestException as e:
            print(f"  -> Erro de conexão ao tentar {url}: {e}")
        except zipfile.BadZipFile:
            print(f"  -> Erro: Arquivo baixado de {url} não é um ZIP válido.")
        except Exception as e:
            print(f"  -> Erro inesperado ao processar {url}: {e}")

    # Se após todas as tentativas o download falhou para a data
    if not download_success:
        print(f"COSIF Prud. ({date}): FALHA no download após tentar todos os sufixos.")

# --- 2. Leitura e Consolidação dos CSVs ---
df_list = []
print(f"\nLendo {len(all_prudencial_csv_paths)} arquivo(s) CSV do COSIF Prudencial...")
for csv_path in sorted(list(set(all_prudencial_csv_paths))):
    print(f"  Lendo: {csv_path.name}")
    try:
        temp_df = pd.read_csv(csv_path, header=3, encoding='latin1', sep=';', decimal=',', dtype={'CNPJ': str}, on_bad_lines='warn')
        file_date = csv_path.parent.name
        temp_df['DATA'] = file_date
        df_list.append(temp_df)
    except Exception as e:
        print(f"    -> Erro ao ler o arquivo {csv_path.name}: {e}")
        
# --- 3. Processamento e Limpeza do DataFrame ---
if not df_list:
    print("\nAVISO: Nenhum dado do COSIF Prudencial foi carregado. O DataFrame final estará vazio.")
    df_cosif_prudencial = pd.DataFrame()
else:
    df_cosif_prudencial = pd.concat(df_list, ignore_index=True)
    print(f"\nShape do DF consolidado (bruto): {df_cosif_prudencial.shape}")

    # Padronização da chave CNPJ_8
    df_cosif_prudencial['CNPJ_8'] = du.standardize_cnpj_base8(df_cosif_prudencial['CNPJ'])
    
    # Padronização de data
    df_cosif_prudencial['DATA'] = pd.to_datetime(df_cosif_prudencial['DATA'], format='%Y%m').dt.strftime('%Y%m').astype(int)

    # Mapa de renomeação para o novo padrão
    rename_map = {
        'NOME_INSTITUICAO': 'NOME_INSTITUICAO_COSIF',
        'CONTA': 'CONTA_COSIF',
        'NOME_CONTA': 'NOME_CONTA_COSIF',
        'SALDO': 'VALOR_CONTA_COSIF',
        'COD_CONGL': 'COD_CONGL_PRUD_COSIF',
        'NOME_CONGL': 'NOME_CONGL_PRUD_COSIF',
        'TAXONOMIA': 'TAXONOMIA_COSIF',
        'DOCUMENTO': 'DOCUMENTO_COSIF',
        'AGENCIA': 'AGENCIA_COSIF'
    }
    df_cosif_prudencial.rename(columns=rename_map, inplace=True)
    
    # Limpeza das colunas de texto (já com os novos nomes)
    for col in ['NOME_INSTITUICAO_COSIF', 'NOME_CONGL_PRUD_COSIF', 'NOME_CONTA_COSIF']:
        if col in df_cosif_prudencial.columns:
            df_cosif_prudencial[col] = clean_text_column(df_cosif_prudencial[col])
    
    # Descarte de colunas redundantes
    cols_to_drop = ['CNPJ', '#DATA_BASE']
    df_cosif_prudencial.drop(columns=[c for c in cols_to_drop if c in df_cosif_prudencial.columns], inplace=True)

    # Reordenar colunas para melhor visualização
    cols_ordem_prioritaria = [
        'DATA', 'CNPJ_8', 'NOME_INSTITUICAO_COSIF', 'CONTA_COSIF', 'NOME_CONTA_COSIF', 
        'VALOR_CONTA_COSIF', 'COD_CONGL_PRUD_COSIF', 'NOME_CONGL_PRUD_COSIF', 
        'TAXONOMIA_COSIF', 'DOCUMENTO_COSIF', 'AGENCIA_COSIF'
    ]
    cols_existentes_prioritarias = [c for c in cols_ordem_prioritaria if c in df_cosif_prudencial.columns]
    cols_restantes = [c for c in df_cosif_prudencial.columns if c not in cols_existentes_prioritarias]
    df_cosif_prudencial = df_cosif_prudencial[cols_existentes_prioritarias + cols_restantes]

    print("\n--- Informações do DataFrame Final: df_cosif_prudencial ---")
    print(f"Shape: {df_cosif_prudencial.shape}")
    print(f"Colunas: {df_cosif_prudencial.columns.tolist()}")
    print(f"Período de dados: {df_cosif_prudencial['DATA'].min()} a {df_cosif_prudencial['DATA'].max()}")

    # --- 4. Salvando o Resultado ---
    output_path = dir_outputs / 'df_cosif_prudencial.parquet'
    df_cosif_prudencial.to_parquet(output_path, index=False)
    print(f"\nArquivo salvo com sucesso em: {output_path}")

In [ ]:
# ==============================================================================
# CÉLULA 4: Pipeline de Dados - IFDATA Valores
# ==============================================================================
# Descrição: Baixa e consolida os dados trimestrais de "Valores" da API IFDATA.
# O resultado final é um único DataFrame salvo em formato Parquet com nomes padronizados.
# ==============================================================================
print("\n>>> INICIANDO PIPELINE: IFDATA VALORES (PODE DEMORAR) <<<")

# --- 1. Download dos Arquivos ---
ifdata_dir = dir_inputs / 'IFDATA'
ifdata_dir.mkdir(parents=True, exist_ok=True)
tipos_instituicao = [1, 2, 3]

MAX_TENTATIVAS = 3
DELAY_SEGUNDOS = 5

for date_q in date_range_quarterly:
    for tipo in tipos_instituicao:
        output_csv_val = ifdata_dir / f"IfDataValores_{date_q}_{tipo}.csv"
        if output_csv_val.exists():
            print(f"IFDATA Valores ({date_q} | Tipo {tipo}): CSV já existe, pulando download.")
            continue

        url_val = (
            f"https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/"
            f"IfDataValores(AnoMes=@AnoMes,TipoInstituicao=@TipoInstituicao,"
            f"Relatorio=@Relatorio)?@AnoMes={date_q}&@TipoInstituicao={tipo}"
            f"&@Relatorio='T'&$format=text/csv"
        )
        print(f"IFDATA Valores ({date_q} | Tipo {tipo}): Baixando…")

        sucesso = False
        for tentativa in range(1, MAX_TENTATIVAS + 1):
            try:
                resp = requests.get(url_val, timeout=120)
                resp.raise_for_status()
                with open(output_csv_val, 'wb') as f:
                    f.write(resp.content)
                print(f"  -> Sucesso na tentativa {tentativa}!")
                sucesso = True
                break
            except requests.exceptions.RequestException as e:
                print(f"  -> Tentativa {tentativa} falhou: {e}")
                if tentativa < MAX_TENTATIVAS:
                    print(f"     esperando {DELAY_SEGUNDOS}s antes da próxima tentativa…")
                    time.sleep(DELAY_SEGUNDOS)

        if not sucesso:
            print(f"  => Todas as {MAX_TENTATIVAS} tentativas falharam para ({date_q} | Tipo {tipo}). Pulando.")

# --- 2. Leitura e Consolidação dos CSVs ---
csv_files = sorted(ifdata_dir.glob("IfDataValores_*.csv"))
df_list = []
print(f"\nLendo {len(csv_files)} arquivo(s) CSV do IFDATA Valores...")

for path in csv_files:
    if path.stat().st_size > 100:
        print(f"  Lendo: {path.name}")
        try:
            temp = pd.read_csv(path, encoding='utf-8', sep=',', decimal='.', dtype={'CodInst': str})
            df_list.append(temp)
        except Exception as e:
            print(f"    -> Erro ao ler o arquivo {path.name}: {e}")

# --- 3. Processamento e Limpeza do DataFrame ---
if not df_list:
    print("\nAVISO: Nenhum dado do IFDATA Valores foi carregado. O DataFrame final estará vazio.")
    df_ifdata_valores = pd.DataFrame()
else:
    df_ifdata_valores = pd.concat(df_list, ignore_index=True)
    print(f"\nShape do DF consolidado (bruto): {df_ifdata_valores.shape}")

    # Mapa de renomeação para o novo padrão
    rename_map = {
        'AnoMes': 'DATA',
        'CodInst': 'COD_INST_IFD_VAL',
        'TipoInstituicao': 'TIPO_INSTITUICAO_IFD_VAL',
        'Conta': 'CONTA_IFD_VAL',
        'NomeColuna': 'NOME_CONTA_IFD_VAL',
        'Saldo': 'VALOR_CONTA_IFD_VAL',
        'NomeRelatorio': 'NOME_RELATORIO_IFD_VAL',
        'NumeroRelatorio': 'NUMERO_RELATORIO_IFD_VAL',
        'Grupo': 'GRUPO_CONTA_IFD_VAL',
        'DescricaoColuna': 'DESCRICAO_CONTA_IFD_VAL'
    }
    df_ifdata_valores.rename(columns=rename_map, inplace=True)
    
    # Conversão e limpeza de tipos de dados
    df_ifdata_valores['DATA'] = df_ifdata_valores['DATA'].astype(int)
    if 'VALOR_CONTA_IFD_VAL' in df_ifdata_valores.columns:
        df_ifdata_valores['VALOR_CONTA_IFD_VAL'] = pd.to_numeric(
            df_ifdata_valores['VALOR_CONTA_IFD_VAL'].astype(str).str.replace(',', '.'), 
            errors='coerce'
        )

    # Limpeza das colunas de texto (já com os novos nomes)
    for col in ['NOME_RELATORIO_IFD_VAL', 'GRUPO_CONTA_IFD_VAL', 'NOME_CONTA_IFD_VAL', 'DESCRICAO_CONTA_IFD_VAL']:
        if col in df_ifdata_valores.columns:
            df_ifdata_valores[col] = clean_text_column(df_ifdata_valores[col])
    
    # Reordenar colunas
    cols_ordem_prioritaria = [
        'DATA', 'COD_INST_IFD_VAL', 'TIPO_INSTITUICAO_IFD_VAL', 'CONTA_IFD_VAL', 'NOME_CONTA_IFD_VAL', 
        'VALOR_CONTA_IFD_VAL', 'NOME_RELATORIO_IFD_VAL', 'NUMERO_RELATORIO_IFD_VAL', 
        'GRUPO_CONTA_IFD_VAL', 'DESCRICAO_CONTA_IFD_VAL'
    ]
    cols_existentes_prioritarias = [c for c in cols_ordem_prioritaria if c in df_ifdata_valores.columns]
    cols_restantes = [c for c in df_ifdata_valores.columns if c not in cols_existentes_prioritarias]
    df_ifdata_valores = df_ifdata_valores[cols_existentes_prioritarias + cols_restantes]

    print("\n--- Informações do DataFrame Final: df_ifdata_valores ---")
    print(f"Shape: {df_ifdata_valores.shape}")
    print(f"Colunas: {df_ifdata_valores.columns.tolist()}")
    print(f"Período de dados: {df_ifdata_valores['DATA'].min()} a {df_ifdata_valores['DATA'].max()}")

    # --- 4. Salvando o Resultado ---
    output_path = dir_outputs / 'df_ifdata_valores.parquet'
    df_ifdata_valores.to_parquet(output_path, index=False)
    print(f"\nArquivo salvo com sucesso em: {output_path}")

In [ ]:
# ==============================================================================
# CÉLULA 5: Pipeline de Dados - IFDATA Cadastro
# ==============================================================================
# Descrição: Baixa e consolida os dados mensais de "Cadastro" da API IFDATA.
# O resultado final é um único DataFrame salvo em formato Parquet com nomes padronizados.
# ==============================================================================
print("\n>>> INICIANDO PIPELINE: IFDATA CADASTRO <<<")

# --- 1. Download dos Arquivos ---
ifdata_dir = dir_inputs / 'IFDATA' 
for date_m in date_range_monthly:
    output_csv_cad = ifdata_dir / f"IfDataCadastro_{date_m}.csv"
    if output_csv_cad.exists():
        print(f"IFDATA Cadastro ({date_m}): CSV já existe, pulando download.")
        continue
            
    url_cadastro = f"https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/IfDataCadastro(AnoMes=@AnoMes)?@AnoMes={date_m}&$format=text/csv"
    print(f"IFDATA Cadastro ({date_m}): Baixando...")
    try:
        resp = requests.get(url_cadastro, timeout=120)
        resp.raise_for_status()
        with open(output_csv_cad, 'wb') as f:
            f.write(resp.content)
        print("  -> Sucesso!")
    except requests.exceptions.RequestException as e:
        print(f"  -> Erro no download: {e}")

# --- 2. Leitura e Consolidação dos CSVs ---
csv_files = sorted(ifdata_dir.glob("IfDataCadastro_*.csv"))
df_list = []
print(f"\nLendo {len(csv_files)} arquivo(s) CSV do IFDATA Cadastro...")
for path in csv_files:
    if path.stat().st_size > 100:
        print(f"  Lendo: {path.name}")
        try:
            temp = pd.read_csv(path, encoding='utf-8', sep=',', decimal='.', dtype={'CodInst': str, 'CnpjInstituicaoLider': str})
            df_list.append(temp)
        except Exception as e:
            print(f"    -> Erro ao ler o arquivo {path.name}: {e}")

# --- 3. Processamento e Limpeza do DataFrame ---
if not df_list:
    print("\nAVISO: Nenhum dado do IFDATA Cadastro foi carregado. O DataFrame final estará vazio.")
    df_ifdata_cadastro = pd.DataFrame()
else:
    df_ifdata_cadastro = pd.concat(df_list, ignore_index=True).drop_duplicates()
    print(f"\nShape do DF consolidado (bruto): {df_ifdata_cadastro.shape}")

    # Padronização das chaves
    df_ifdata_cadastro['CNPJ_8'] = du.standardize_cnpj_base8(df_ifdata_cadastro['CodInst'])
    df_ifdata_cadastro['CNPJ_LIDER_8_IFD_CAD'] = du.standardize_cnpj_base8(df_ifdata_cadastro['CnpjInstituicaoLider'])

    # Renomeação inicial e padronização de data
    df_ifdata_cadastro.rename(columns={'Data': 'DATA'}, inplace=True)
    df_ifdata_cadastro['DATA'] = df_ifdata_cadastro['DATA'].astype(int)

    # Renomear TODAS as colunas restantes com o sufixo _IFD_CAD
    cols_to_rename = [col for col in df_ifdata_cadastro.columns if col not in ['DATA', 'CNPJ_8', 'CNPJ_LIDER_8_IFD_CAD', 'CodInst', 'CnpjInstituicaoLider']]
    rename_map = {col: f"{col.upper()}_IFD_CAD" for col in cols_to_rename}
    
    # Renomear as colunas principais do cadastro para o padrão
    rename_map.update({
        'NomeInstituicao': 'NOME_INSTITUICAO_IFD_CAD',
        'CodConglomeradoPrudencial': 'COD_CONGL_PRUD_IFD_CAD',
        'CodConglomeradoFinanceiro': 'COD_CONGL_FIN_IFD_CAD',
        'DataInicioAtividade': 'DATA_INICIO_ATIVIDADE_IFD_CAD',
    })
    
    df_ifdata_cadastro.rename(columns=rename_map, inplace=True)

    # Limpeza de colunas de texto (já com os novos nomes)
    text_cols = ['NOME_INSTITUICAO_IFD_CAD', 'SEGMENTOTB_IFD_CAD', 'ATIVIDADE_IFD_CAD', 'UF_IFD_CAD', 'MUNICIPIO_IFD_CAD', 'SITUACAO_IFD_CAD']
    for col in text_cols:
        if col in df_ifdata_cadastro.columns:
            df_ifdata_cadastro[col] = clean_text_column(df_ifdata_cadastro[col])
            
    # Descarte de colunas redundantes
    cols_to_drop = ['CodInst', 'CnpjInstituicaoLider']
    df_ifdata_cadastro.drop(columns=[c for c in cols_to_drop if c in df_ifdata_cadastro.columns], inplace=True)

    # Reordenar colunas
    cols_ordem_prioritaria = [
        'DATA', 'CNPJ_8', 'NOME_INSTITUICAO_IFD_CAD', 'COD_CONGL_PRUD_IFD_CAD',
        'CNPJ_LIDER_8_IFD_CAD', 'COD_CONGL_FIN_IFD_CAD', 'SITUACAO_IFD_CAD',
        'DATA_INICIO_ATIVIDADE_IFD_CAD', 'SEGMENTOTB_IFD_CAD', 'ATIVIDADE_IFD_CAD',
        'TCB_IFD_CAD', 'TD_IFD_CAD', 'TC_IFD_CAD', 'UF_IFD_CAD', 'MUNICIPIO_IFD_CAD', 'SR_IFD_CAD'
    ]
    cols_existentes_prioritarias = [c for c in cols_ordem_prioritaria if c in df_ifdata_cadastro.columns]
    cols_restantes = [c for c in df_ifdata_cadastro.columns if c not in cols_existentes_prioritarias]
    df_ifdata_cadastro = df_ifdata_cadastro[cols_existentes_prioritarias + cols_restantes]

    print("\n--- Informações do DataFrame Final: df_ifdata_cadastro ---")
    print(f"Shape: {df_ifdata_cadastro.shape}")
    print(f"Colunas: {df_ifdata_cadastro.columns.tolist()}")
    print(f"Período de dados: {df_ifdata_cadastro['DATA'].min()} a {df_ifdata_cadastro['DATA'].max()}")
    
    # --- 4. Salvando o Resultado ---
    output_path = dir_outputs / 'df_ifdata_cadastro.parquet'
    df_ifdata_cadastro.to_parquet(output_path, index=False)
    print(f"\nArquivo salvo com sucesso em: {output_path}")

In [ ]:
# ==============================================================================
# CÉLULA 6: Geração de Saídas Auxiliares (Mapeamento, Infos e Dicionários)
# ==============================================================================
# Descrição: Cria arquivos secundários úteis para a análise, já adaptados
# para a nova estrutura de nomes de colunas e com dicionários COSIF separados.
# ==============================================================================
print("\n>>> GERANDO SAÍDAS AUXILIARES <<<")

# --- 1. Criar DataFrame de Mapeamento (CNPJ -> Conglomerado) ---
try:
    if not df_cosif_prudencial.empty:
        print("\nCriando mapeamento CNPJ -> Conglomerado...")
        df_mapeamento = (
            df_cosif_prudencial[['CNPJ_8', 'COD_CONGL_PRUD_COSIF', 'NOME_CONGL_PRUD_COSIF', 'DATA']]
            .sort_values('DATA', ascending=False)
            .drop_duplicates(subset=['CNPJ_8'], keep='first')
            .drop(columns='DATA')
            .reset_index(drop=True)
            .rename(columns={
                'COD_CONGL_PRUD_COSIF': 'COD_CONGL_PRUD',
                'NOME_CONGL_PRUD_COSIF': 'NOME_CONGL_PRUD'
            })
        )
        
        output_path_map = dir_outputs / 'df_mapeamento_cnpj_conglomerado.parquet'
        df_mapeamento.to_parquet(output_path_map, index=False)
        print(f"Shape do mapeamento: {df_mapeamento.shape}")
        print(f" -> Sucesso! Arquivo de mapeamento salvo em: {output_path_map}")
    else:
        print("\nAVISO: df_cosif_prudencial está vazio. O arquivo de mapeamento não será gerado.")
except NameError:
    print("\nAVISO: Variável 'df_cosif_prudencial' não foi criada. O arquivo de mapeamento não será gerado.")


# --- 2. Gerar Arquivos de Informações dos DataFrames (Info) ---
print("\n--- Gerando Arquivos de Informações (Perfil) dos DataFrames ---")
dfs_to_document = {
    'df_cosif_individual': 'cosif_individual',
    'df_cosif_prudencial': 'cosif_prudencial',
    'df_ifdata_valores': 'ifdata_valores',
    'df_ifdata_cadastro': 'ifdata_cadastro'
}

for df_var_name, filename_prefix in dfs_to_document.items():
    if df_var_name in locals() and not locals()[df_var_name].empty:
        df_to_process = locals()[df_var_name]
        info_df = create_data_infos(df_to_process, filename_prefix)
        
        output_excel_path = dir_outputs / f'info_dataframe_{filename_prefix}.xlsx'
        info_df.to_excel(output_excel_path, index=False, engine='openpyxl')
        print(f" -> Sucesso! Arquivo de info salvo em: {output_excel_path}\n")
    else:
        print(f"\nAVISO: DataFrame '{df_var_name}' está vazio ou não existe. Arquivo de info não será gerado.")


# --- 3. Gerar Dicionários de Contas (Código -> Nome) ---
print("\n--- Gerando Dicionários de Contas ---")

# MODIFICAÇÃO: Dicionários COSIF agora são gerados separadamente.

# Dicionário para COSIF Individual
print("\nProcessando dicionário para: COSIF Individual")
try:
    if not df_cosif_individual.empty:
        dict_cosif_ind = (
            df_cosif_individual[['CONTA_COSIF', 'NOME_CONTA_COSIF']]
            .drop_duplicates().sort_values('CONTA_COSIF').reset_index(drop=True)
        )
        path = dir_outputs / 'dicionario_contas_cosif_individual.xlsx'
        dict_cosif_ind.to_excel(path, index=False)
        print(f" -> Sucesso! Dicionário salvo em: {path}")
    else:
        print(" -> AVISO: df_cosif_individual vazio ou colunas de conta não encontradas.")
except NameError:
    print(" -> AVISO: DataFrame 'df_cosif_individual' não existe. Dicionário não gerado.")

# Dicionário para COSIF Prudencial
print("\nProcessando dicionário para: COSIF Prudencial")
try:
    if not df_cosif_prudencial.empty:
        dict_cosif_prud = (
            df_cosif_prudencial[['CONTA_COSIF', 'NOME_CONTA_COSIF']]
            .drop_duplicates().sort_values('CONTA_COSIF').reset_index(drop=True)
        )
        path = dir_outputs / 'dicionario_contas_cosif_prudencial.xlsx'
        dict_cosif_prud.to_excel(path, index=False)
        print(f" -> Sucesso! Dicionário salvo em: {path}")
    else:
        print(" -> AVISO: df_cosif_prudencial vazio ou colunas de conta não encontradas.")
except NameError:
    print(" -> AVISO: DataFrame 'df_cosif_prudencial' não existe. Dicionário não gerado.")

# Dicionário para IFDATA Valores
print("\nProcessando dicionário para: IFDATA Valores")
try:
    if not df_ifdata_valores.empty:
        dict_ifdata = (
            df_ifdata_valores[['CONTA_IFD_VAL', 'NOME_CONTA_IFD_VAL']]
            .drop_duplicates().sort_values('CONTA_IFD_VAL').reset_index(drop=True)
        )
        path = dir_outputs / 'dicionario_contas_ifdata_valores.xlsx'
        dict_ifdata.to_excel(path, index=False)
        print(f" -> Sucesso! Dicionário salvo em: {path}")
    else:
        print(" -> AVISO: df_ifdata_valores vazio ou colunas de conta não encontradas.")
except NameError:
    print(" -> AVISO: DataFrame 'df_ifdata_valores' não existe. Dicionário não gerado.")

# --- 4. Gerar Dicionário de Entidades (Nomes, CNPJs e Códigos) ---
print("\n--- Gerando Dicionário de Entidades para Consulta ---")

try:
    # Passo 1: Criar a base de dados de entidades a partir do IFDATA Cadastro, incluindo todos os identificadores
    # Selecionamos a entrada mais recente para cada CNPJ_8 para obter os códigos mais atuais.
    df_base_entidades = (
        df_ifdata_cadastro[[
            'DATA', 'CNPJ_8', 'NOME_INSTITUICAO_IFD_CAD', 
            'COD_CONGL_PRUD_IFD_CAD', 'COD_CONGL_FIN_IFD_CAD', 'CNPJ_LIDER_8_IFD_CAD'
        ]]
        .sort_values('DATA', ascending=False)
        .drop_duplicates(subset=['CNPJ_8'], keep='first')
        .drop(columns='DATA')
        .rename(columns={
            'NOME_INSTITUICAO_IFD_CAD': 'NOME_PRINCIPAL',
            'COD_CONGL_PRUD_IFD_CAD': 'COD_CONGL_PRUD',
            'COD_CONGL_FIN_IFD_CAD': 'COD_CONGL_FIN',
            'CNPJ_LIDER_8_IFD_CAD': 'CNPJ_LIDER'
        })
    ).dropna(subset=['CNPJ_8'])

    # Passo 2: Coletar todos os nomes únicos e seus CNPJs de todas as fontes
    nomes_cosif_prud = df_cosif_prudencial[['CNPJ_8', 'NOME_INSTITUICAO_COSIF']].rename(columns={'NOME_INSTITUICAO_COSIF': 'NOME_VARIACOES'})
    nomes_cosif_ind = df_cosif_individual[['CNPJ_8', 'NOME_INSTITUICAO_COSIF']].rename(columns={'NOME_INSTITUICAO_COSIF': 'NOME_VARIACOES'})
    nomes_ifd_cad = df_ifdata_cadastro[['CNPJ_8', 'NOME_INSTITUICAO_IFD_CAD']].rename(columns={'NOME_INSTITUICAO_IFD_CAD': 'NOME_VARIACOES'})
    
    df_todos_nomes = (
        pd.concat([nomes_cosif_prud, nomes_cosif_ind, nomes_ifd_cad])
        .dropna()
        .drop_duplicates()
        .groupby('CNPJ_8')['NOME_VARIACOES']
        .apply(lambda x: ' | '.join(sorted(list(set(x)))))
        .reset_index()
    )

    # Passo 3: Fazer o merge da base de entidades com a lista de todos os nomes
    df_entidades_final = pd.merge(
        df_base_entidades,
        df_todos_nomes,
        on='CNPJ_8',
        how='outer'
    )
    
    # Preencher o nome principal caso ele esteja faltando (para entidades que só existem no COSIF)
    df_entidades_final['NOME_PRINCIPAL'] = df_entidades_final['NOME_PRINCIPAL'].fillna(
        df_entidades_final['NOME_VARIACOES'].str.split(' | ').str[0]
    )
    
    # Reordenar as colunas para uma apresentação lógica
    cols_ordem = [
        'NOME_PRINCIPAL', 
        'CNPJ_8', 
        'CNPJ_LIDER', 
        'COD_CONGL_PRUD', 
        'COD_CONGL_FIN', 
        'NOME_VARIACOES'
    ]
    df_entidades_final = df_entidades_final[cols_ordem]
    df_entidades_final = df_entidades_final.sort_values('NOME_PRINCIPAL').reset_index(drop=True)

    # Salvar em Excel
    output_path_entidades = dir_outputs / 'dicionario_entidades.xlsx'
    df_entidades_final.to_excel(output_path_entidades, index=False)
    
    print(f"-> {len(df_entidades_final)} entidades únicas salvas com sucesso em: {output_path_entidades}")
    print("O dicionário agora contém todos os identificadores chave (Líder, Prudencial, Financeiro).")

except Exception as e:
    print(f"ERRO ao gerar o dicionário de entidades: {e}")